In [2]:
import numpy as np
import pandas as pd

from pathlib import Path

In [3]:
data_pth = Path('./data')

In [8]:
dtypes = {
    'fare_amount': np.float16,
    'pickup_datetime': 'str',
    'pickup_longitude': 'float',
    'pickup_latitude': 'float',
    'dropoff_longitude': 'float',
    'dropoff_latitude': 'float',
    'passenger_count': np.int16
}
use_cols = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude', 
    'dropoff_longitude', 
    'dropoff_latitude',
    'passenger_count'
]
date_cols = ['pickup_datetime']
pd.read_csv(data_pth/'train.csv', nrows=20, dtype=dtypes, use_cols=use_cols, parse_dates=date_cols, )

TypeError: parser_f() got an unexpected keyword argument 'use_cols'